In [53]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import view_points, BoxVisibility
from pyquaternion import Quaternion
import numpy as np
import pandas as pd
import tqdm

In [16]:
import os

In [18]:
os.chdir("../")

In [83]:
# Loading the nuscene from local system
nusc = NuScenes(version='v1.0-mini', dataroot='data/input_data/v1.0-mini/', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 1.895 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [4]:
my_scene = nusc.scene[0]
first_sample_token = my_scene['first_sample_token']
my_sample = nusc.get('sample', first_sample_token)

In [5]:
sensor = 'CAM_FRONT'
cam_front_data_token = my_sample['data'][sensor]
cam_front_data = nusc.get('sample_data', cam_front_data_token)
cam_front_data

{'token': 'e3d495d4ac534d54b321f50006683844',
 'sample_token': 'ca9a282c9e77460f8360f564131a8af5',
 'ego_pose_token': 'e3d495d4ac534d54b321f50006683844',
 'calibrated_sensor_token': '1d31c729b073425e8e0202c5c6e66ee1',
 'timestamp': 1532402927612460,
 'fileformat': 'jpg',
 'is_key_frame': True,
 'height': 900,
 'width': 1600,
 'filename': 'samples/CAM_FRONT/n015-2018-07-24-11-22-45+0800__CAM_FRONT__1532402927612460.jpg',
 'prev': '',
 'next': '68e8e98cf7b0487baa139df808641db7',
 'sensor_modality': 'camera',
 'channel': 'CAM_FRONT'}

In [212]:
channel_names = ["CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT",
                "CAM_BACK", "CAM_BACK_LEFT", "CAM_BACK_RIGHT"]

image_data_dict = dict()
# Iterate to scene

for scene in tqdm.tqdm(nusc.scene):
    #Iterate to sample
    token = scene['first_sample_token']
    while token != '':
        sample = nusc.get('sample', token)
        annotation_list = sample['anns']
        
        for channel in channel_names:
            channel_token = sample['data'][channel]
            filename, boxes, camera_intrinsic = nusc.get_sample_data(channel_token, selected_anntokens=annotation_list)
            
            if channel_token not in image_data_dict:
                image_data_dict[channel_token] = {'channel':channel,
                                                 'iID': camera_token,
                                                         'filePath': filename,
                                                         'annotations': []}
            for box in boxes:
                corners = view_points(box.corners(), np.array(camera_intrinsic), normalize=True)[:2, :]
                image_data_dict[channel_token]['annotations'].append({'name': box.name, 'corners': corners.tolist()})

        token = sample['next']
#         print(token)
#         break
        

100%|██████████| 10/10 [00:10<00:00,  1.07s/it]


## generate class distribution

In [219]:
# with open('data/output_data/camera_boxes.json', 'r') as file:
#     data = json.load(file)

In [221]:
channel_list, iID_list, name_list, corners_list = [], [], [], []
# data = image_data_dict.copy()

for key, value in data.items():
    iID = value["iID"]
    annotations = value["annotations"]
    channel = value["channel"]
    
    for annotation in annotations:
        name = annotation["name"]
        corners = annotation["corners"]
        
        # Append the data to the respective lists
        iID_list.append(iID)
        name_list.append(name)
        channel_list.append(channel)
        corners_list.append(corners)

# Create a Pandas DataFrame from the lists
df = pd.DataFrame({
    "channel": channel_list,
    "iID": iID_list,
    "name": name_list,
    "corners": corners_list
})

In [222]:
df[['class_name', 'sub_category', 'grain']] = df['name'].str.split('.', expand=True, n=2)

In [223]:
class_counts = pd.DataFrame({'counts': df.class_name.value_counts(),
                                 'proportion': df.class_name.value_counts(normalize=True)
                                }).reset_index().rename(columns={"index":"class_name"})
class_counts = class_counts[['class_name', 'counts', 'proportion']]
class_counts

class_name  counts  proportion
0         vehicle   11929    0.540458
1           human    5701    0.258291
2  movable_object    4373    0.198124
3   static_object      69    0.003126

In [224]:
sub_category_counts = pd.DataFrame({'counts': df[['class_name', 'sub_category']].value_counts(),
                                 'proportion': df[['class_name', 'sub_category']].value_counts(normalize=True)
                                }).reset_index()
sub_category_counts = sub_category_counts[['class_name', 'sub_category', 'counts', 'proportion']]
sub_category_counts

class_name       sub_category  counts  proportion
0          vehicle                car    9427    0.427102
1            human         pedestrian    5701    0.258291
2   movable_object            barrier    2692    0.121964
3   movable_object        trafficcone    1574    0.071312
4          vehicle              truck     851    0.038556
5          vehicle         motorcycle     543    0.024601
6          vehicle                bus     526    0.023831
7          vehicle            bicycle     274    0.012414
8          vehicle       construction     245    0.011100
9   movable_object  pushable_pullable      91    0.004123
10   static_object       bicycle_rack      69    0.003126
11         vehicle            trailer      63    0.002854
12  movable_object             debris      16    0.000725

In [287]:
sub_category_counts['counts'].sum()

22072

In [225]:
image_data = pd.read_csv('data/output_data/v1.0-trainval_image_data.csv')
image_data.shape

(1405967, 6)

In [226]:
image_data['channel'].value_counts()

CAM_BACK           389416
CAM_FRONT          331889
CAM_FRONT_LEFT     192705
CAM_FRONT_RIGHT    182647
CAM_BACK_LEFT      162110
CAM_BACK_RIGHT     147200
Name: channel, dtype: int64

In [288]:
class_counts = pd.read_csv('data/output_data/v1.0-trainval_class_counts.csv')
class_counts

class_name       sub_category  counts  proportion
0          vehicle                car  138944    0.418844
1            human         pedestrian   62131    0.187293
2   movable_object            barrier   45062    0.135839
3          vehicle              truck   25673    0.077391
4   movable_object        trafficcone   25227    0.076046
5   movable_object  pushable_pullable    8437    0.025433
6          vehicle            trailer    7276    0.021933
7          vehicle                bus    5714    0.017225
8          vehicle       construction    4709    0.014195
9          vehicle         motorcycle    3304    0.009960
10         vehicle            bicycle    2960    0.008923
11  movable_object             debris    1173    0.003536
12   static_object       bicycle_rack     911    0.002746
13         vehicle          emergency     211    0.000636

## leaderboard

In [231]:
with open('data/input_data/leaderboard.json', 'r') as file:
    leader_data = json.load(file)

In [280]:
leader_data[0]['metrics_summary'].keys()

dict_keys(['label_aps', 'mean_dist_aps', 'mean_ap', 'label_tp_errors', 'tp_errors', 'tp_scores', 'nd_score', 'eval_time', 'cfg'])

In [283]:
leader_data[0]['metrics_summary']['nd_score']

0.7761520312130207

In [265]:
mean_scores = {}
for i in range(5):
    # Calculate mean performance score of top 5 submissions
    metric_summary = leader_data[i]['metrics_summary']['mean_dist_aps']
    for key, value in metric_summary.items():
        if key in mean_scores:
            mean_scores[key].append(value)
        else:
            mean_scores[key] = [value]

# Calculate the mean for each object
for key, value in mean_scores.items():
    mean_scores[key] = sum(value) / len(value)

# Create a DataFrame from the mean scores
leader_df = pd.DataFrame(mean_scores.items(), columns=['sub_category', 'mean_aps_score'])
leader_df

sub_category  mean_aps_score
0                   car        0.899901
1                 truck        0.676586
2                   bus        0.755723
3               trailer        0.686522
4  construction_vehicle        0.442497
5            pedestrian        0.917354
6            motorcycle        0.839903
7               bicycle        0.666053
8          traffic_cone        0.893431
9               barrier        0.809650

In [276]:
class_counts.loc[class_counts['sub_category']=='construction', 'sub_category'] = 'construction_vehicle'
class_counts.loc[class_counts['sub_category']=='trafficcone', 'sub_category'] = 'traffic_cone'

In [285]:
final_metrics_df = class_counts.merge(leader_df, on = 'sub_category', how='inner')
final_metrics_df.rename(columns = {'class_name': 'class_group', 'sub_category': 'class_name'}, inplace=True)

In [286]:
final_metrics_df

class_group            class_name  counts  proportion  mean_aps_score
0         vehicle                   car  611816    0.435436        0.899901
1           human            pedestrian  255160    0.181600        0.917354
2  movable_object               barrier  181597    0.129245        0.809650
3         vehicle                 truck  111392    0.079279        0.676586
4  movable_object          traffic_cone  109313    0.077799        0.893431
5         vehicle               trailer   33277    0.023684        0.686522
6         vehicle                   bus   19773    0.014073        0.755723
7         vehicle  construction_vehicle   19245    0.013697        0.442497
8         vehicle            motorcycle   14640    0.010419        0.839903
9         vehicle               bicycle   13934    0.009917        0.666053